<a href="https://colab.research.google.com/github/lethanhtu-2k5/ttnt/blob/main/ThuattoanAKTvaAsao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import abc
import copy
from heapq import heappush, heappop


# ============================
# 1) LỚP TRỪU TƯỢNG A
# ============================

class A(abc.ABC):
    n = 3

    # 4 hướng di chuyển
    rows = [1, 0, -1, 0]
    cols = [0, -1, 0, 1]

    def __init__(self, initial, final, empty_tile_pos):
        self.initial = initial
        self.final = final
        self.empty_tile_pos = empty_tile_pos

    # ----------------------------
    class Node:
        def __init__(self, parent, mats, empty_pos, g, h):
            self.parent = parent
            self.mats = mats
            self.empty_pos = empty_pos
            self.g = g
            self.h = h
            self.f = g + h

        def __lt__(self, other):
            return self.f < other.f
    # ----------------------------

    # Hàm in ma trận
    def printMatrix(self, mats):
        for i in range(self.n):
            for j in range(self.n):
                print(mats[i][j], end=" ")
            print()
        print()

    # Hàm in đường đi bằng đệ quy từ gốc → node hiện tại
    def printPath(self, node):
        if node is None:
            return
        self.printPath(node.parent)
        self.printMatrix(node.mats)

    # Kiểm tra di chuyển hợp lệ
    def isSafe(self, x, y):
        return 0 <= x < self.n and 0 <= y < self.n

    # Heuristic mặc định: số ô sai vị trí (dùng cho AKT)
    def misplacedTiles(self, mats):
        cnt = 0
        for i in range(self.n):
            for j in range(self.n):
                if mats[i][j] != 0 and mats[i][j] != self.final[i][j]:
                    cnt += 1
        return cnt

    # Heuristic Manhattan (dùng cho AStar)
    def manhattan(self, mats):
        pos = {}
        for i in range(self.n):
            for j in range(self.n):
                pos[self.final[i][j]] = (i, j)

        total = 0
        for i in range(self.n):
            for j in range(self.n):
                val = mats[i][j]
                if val != 0:
                    gx, gy = pos[val]
                    total += abs(gx - i) + abs(gy - j)
        return total

    # Tạo node con sau di chuyển
    def newNode(self, mats, old_pos, new_pos, g, parent, heuristic_type="misplaced"):
        new_mats = copy.deepcopy(mats)
        x1, y1 = old_pos
        x2, y2 = new_pos

        # Hoán đổi ô trống
        new_mats[x1][y1], new_mats[x2][y2] = new_mats[x2][y2], new_mats[x1][y1]

        # Tính h
        if heuristic_type == "misplaced":
            h = self.misplacedTiles(new_mats)
        else:
            h = self.manhattan(new_mats)

        return self.Node(parent, new_mats, new_pos, g, h)

    # PHƯƠNG THỨC GIẢI — MỖI LỚP CON PHẢI OVERRIDE
    @abc.abstractmethod
    def solve(self):
        pass


# ============================
# 2) LỚP CON AKT
# ============================

class AKT(A):
    def solve(self):
        print("=== Solving with AKT (Branch & Bound) ===")

        pq = []
        root = self.Node(None, self.initial, self.empty_tile_pos,
                         g=0,
                         h=self.misplacedTiles(self.initial))
        heappush(pq, root)

        visited = set()

        while pq:
            current = heappop(pq)
            mats_tuple = tuple(tuple(row) for row in current.mats)

            if mats_tuple in visited:
                continue
            visited.add(mats_tuple)

            # FOUND GOAL
            if current.h == 0:
                print("=== Solution Found by AKT ===")
                self.printPath(current)
                print("Moves:", current.g)
                return

            # Sinh node con
            for d in range(4):
                nx = current.empty_pos[0] + self.rows[d]
                ny = current.empty_pos[1] + self.cols[d]

                if self.isSafe(nx, ny):
                    child = self.newNode(current.mats,
                                         current.empty_pos,
                                         [nx, ny],
                                         current.g + 1,
                                         current,
                                         heuristic_type="misplaced")
                    heappush(pq, child)

        print("No solution.")


# ============================
# 3) LỚP CON A*
# ============================

class AStar(A):

    def solve(self):
        print("=== Solving with A* ===")

        pq = []
        root = self.Node(None, self.initial, self.empty_tile_pos,
                         g=0,
                         h=self.manhattan(self.initial))
        heappush(pq, root)

        visited = set()

        while pq:
            current = heappop(pq)
            mats_tuple = tuple(tuple(row) for row in current.mats)

            if mats_tuple in visited:
                continue
            visited.add(mats_tuple)

            # FOUND GOAL
            if current.h == 0:
                print("=== Solution Found by A* ===")
                self.printPath(current)
                print("Moves:", current.g)
                return

            # Sinh node con
            for d in range(4):
                nx = current.empty_pos[0] + self.rows[d]
                ny = current.empty_pos[1] + self.cols[d]

                if self.isSafe(nx, ny):
                    child = self.newNode(current.mats,
                                         current.empty_pos,
                                         [nx, ny],
                                         current.g + 1,
                                         current,
                                         heuristic_type="manhattan")
                    heappush(pq, child)

        print("No solution.")


# ============================
# 4) MAIN TEST
# ============================

initial = [
    [1, 2, 3],
    [4, 5, 0],
    [7, 8, 6]
]

final = [
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 0]
]

empty_pos = [1, 2]


print("\n----- RUN AKT -----\n")
AKT(initial, final, empty_pos).solve()

print("\n----- RUN A* -----\n")
AStar(initial, final, empty_pos).solve()



----- RUN AKT -----

=== Solving with AKT (Branch & Bound) ===
=== Solution Found by AKT ===
1 2 3 
4 5 0 
7 8 6 

1 2 3 
4 5 6 
7 8 0 

Moves: 1

----- RUN A* -----

=== Solving with A* ===
=== Solution Found by A* ===
1 2 3 
4 5 0 
7 8 6 

1 2 3 
4 5 6 
7 8 0 

Moves: 1
